In [ ]:
!uv pip install accelerate transformers -U

In [ ]:
import requests
import time
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor


path = "../input/zebra_on_tree.jpg"
image = Image.open(path)

model_id = "unsloth/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

prompt = "Fill the '...' with the correct values in json format, if you don't know the exact value, use 'None', SFW only:"
prompt += """
{
    "long prompt": "...",# 2-3 sentences
    "short description": "...",
    "background view description": "...",
    "objects": ["...", "..."],
    "principal colors": ["...", "..."],
    "medium": "...",# photography, painting, etc
    "mood": "...",# calm, dramatic, etc
    "overall rating": "...",# 1-10
}
"""


messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": f"{prompt}"}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

for i in range(0,4):
    start_perf = time.perf_counter()
    inputs = processor(image, input_text, return_tensors="pt").to(model.device)

    output = model.generate(**inputs, max_new_tokens=350)
    #print(path)
    print(processor.decode(output[0]))
    stop_perf = time.perf_counter()
    print(f"      ** elapsed: {stop_perf - start_perf:.2f}")